In [1]:
# Indian Classical Raga Sequence Generator using RNN
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import random

# -------------------- Step 1: Raga Definitions -------------------- #
raga_dict = {
    "Default": ['Sa', 'Re', 'Ga', 'Ma', 'Pa', 'Dha', 'Ni', 'Sha'],
    "Bhairav": ['Sa', 'Reb', 'Ga', 'Ma', 'Pa', 'Dhab', 'Ni'],  # komal Re, komal Dha
    "Bhopali": ['Sa', 'Re', 'Ga', 'Pa', 'Dha'],
    "Bageshree": ['Ni', 'Sa', 'Gab', 'Ma', 'Dha', 'Ni']  # komal Ga
}

# -------------------- Step 2: Dataset Generation -------------------- #
def generate_sequences(notes, num_sequences=1000, sequence_length=10):
    sequences = []
    for _ in range(num_sequences):
        seq = [random.choice(notes) for _ in range(sequence_length)]
        sequences.append(seq)
    return sequences

# -------------------- Step 3: Data Preprocessing -------------------- #
def preprocess_sequences(sequences, note_to_int):
    encoded_sequences = [[note_to_int[note] for note in seq] for seq in sequences]
    X, y = [], []
    for seq in encoded_sequences:
        for i in range(len(seq) - 1):
            X.append(seq[:i+1])
            y.append(seq[i+1])
    X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='pre')
    return np.array(X), np.array(y)

# -------------------- Step 4: Model Building -------------------- #
def build_rnn_model(vocab_size, input_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=10, input_length=input_length),
        SimpleRNN(64, return_sequences=False),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# -------------------- Step 5: Sequence Generation -------------------- #
def generate_note_sequence(model, seed_notes, note_to_int, int_to_note, max_len, length=10):
    generated = seed_notes[:]
    for _ in range(length):
        encoded = [note_to_int[n] for n in generated if n in note_to_int]
        padded = tf.keras.preprocessing.sequence.pad_sequences([encoded], maxlen=max_len, padding='pre')
        pred_index = np.argmax(model.predict(padded, verbose=0), axis=-1)[0]
        generated.append(int_to_note[pred_index])
    return generated

# -------------------- Run For Each Raga -------------------- #
for raga_name, notes in raga_dict.items():
    print(f"\n--- Training model for Raga: {raga_name} ---")

    # Mapping
    note_to_int = {note: i for i, note in enumerate(notes)}
    int_to_note = {i: note for note, i in note_to_int.items()}

    # Generate Data
    sequences = generate_sequences(notes)
    X, y = preprocess_sequences(sequences, note_to_int)

    # Build and Train Model
    model = build_rnn_model(len(notes), X.shape[1])
    model.fit(X, y, epochs=10, batch_size=64, verbose=0)

    # Generate Sequence
    seed = sequences[0][:3]
    generated_seq = generate_note_sequence(model, seed, note_to_int, int_to_note, X.shape[1], length=10)
    print("Seed:", seed)
    print("Generated Sequence:", generated_seq)

print("\nAll Ragas processed successfully!")



--- Training model for Raga: Default ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Seed: ['Sa', 'Dha', 'Ga']
Generated Sequence: ['Sa', 'Dha', 'Ga', 'Pa', 'Re', 'Ma', 'Sha', 'Pa', 'Ga', 'Sa', 'Re', 'Pa', 'Sha']

--- Training model for Raga: Bhairav ---
Seed: ['Dhab', 'Ma', 'Ga']
Generated Sequence: ['Dhab', 'Ma', 'Ga', 'Ma', 'Ma', 'Ga', 'Dhab', 'Ni', 'Pa', 'Pa', 'Pa', 'Pa', 'Reb']

--- Training model for Raga: Bhopali ---
Seed: ['Pa', 'Sa', 'Pa']
Generated Sequence: ['Pa', 'Sa', 'Pa', 'Ga', 'Dha', 'Pa', 'Ga', 'Dha', 'Pa', 'Dha', 'Pa', 'Re', 'Re']

--- Training model for Raga: Bageshree ---
Seed: ['Ni', 'Dha', 'Ni']
Generated Sequence: ['Ni', 'Dha', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni', 'Ni']

All Ragas processed successfully!


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
import random

# Define the basic raga notes
notes = ['Sa', 'Re', 'Ga', 'Ma', 'Pa', 'Dha', 'Ni', 'Sha']

# Map notes to integers
note_to_int = {note: i for i, note in enumerate(notes)}
int_to_note = {i: note for note, i in note_to_int.items()}

# Generate synthetic dataset (random sequences of notes)
def generate_sequences(num_sequences=1000, sequence_length=10):
    sequences = []
    for _ in range(num_sequences):
        seq = [random.choice(notes) for _ in range(sequence_length)]
        sequences.append(seq)
    return sequences

# Generate and prepare the dataset
sequences = generate_sequences()


In [4]:
# Convert sequences to integer format
encoded_sequences = [[note_to_int[note] for note in seq] for seq in sequences]

# Prepare input and output
X = []
y = []

for seq in encoded_sequences:
    for i in range(len(seq) - 1):
        X.append(seq[:i+1])
        y.append(seq[i+1])

# Padding sequences to have the same input length
X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='pre')
y = np.array(y)

# Convert to numpy arrays
X = np.array(X)


In [5]:
vocab_size = len(notes)
embedding_dim = 10

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=X.shape[1]),
    SimpleRNN(64, return_sequences=False),
    Dense(vocab_size, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_4 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(X, y, epochs=30, batch_size=64)


Epoch 1/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.1205 - loss: 2.0814
Epoch 2/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1396 - loss: 2.0764
Epoch 3/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1326 - loss: 2.0759
Epoch 4/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1525 - loss: 2.0711
Epoch 5/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1545 - loss: 2.0703
Epoch 6/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1568 - loss: 2.0693
Epoch 7/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1497 - loss: 2.0661
Epoch 8/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1617 - loss: 2.0653
Epoch 9/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1555 - loss: 2.0678
Epoch 10/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1659 - loss: 2.0625
Epoch 11/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.1599 - loss: 2.0652
Epoch 12/30
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step

In [7]:
def generate_note_sequence(seed_notes, length=10):
    generated = seed_notes[:]
    for _ in range(length):
        encoded = [note_to_int[note] for note in generated]
        padded = tf.keras.preprocessing.sequence.pad_sequences([encoded], maxlen=X.shape[1], padding='pre')
        pred_index = np.argmax(model.predict(padded, verbose=0), axis=-1)[0]
        generated.append(int_to_note[pred_index])
    return generated


In [8]:
seed = ['Sa', 'Re', 'Ga']
generated_sequence = generate_note_sequence(seed, length=10)
print("Generated Raga Sequence:", generated_sequence)


Generated Raga Sequence: ['Sa', 'Re', 'Ga', 'Pa', 'Dha', 'Ni', 'Ni', 'Sa', 'Sha', 'Re', 'Ma', 'Ma', 'Ni']


In [11]:
notes_bhopali = ['Sa', 'Re', 'Ga', 'Pa', 'Dha']
note_to_int = {note: i for i, note in enumerate(notes_bhopali)}
int_to_note = {i: note for note, i in note_to_int.items()}
sequences = generate_sequences(num_sequences=5, sequence_length=10)
# Re-run preprocessing, model building, training and generation
sequences

[['Pa', 'Pa', 'Ma', 'Sa', 'Pa', 'Ma', 'Ga', 'Ni', 'Pa', 'Pa'],
 ['Dha', 'Ga', 'Sha', 'Pa', 'Re', 'Ga', 'Sa', 'Ga', 'Dha', 'Pa'],
 ['Ma', 'Re', 'Re', 'Re', 'Pa', 'Dha', 'Ma', 'Ga', 'Ga', 'Pa'],
 ['Pa', 'Re', 'Ga', 'Sa', 'Sha', 'Sa', 'Ni', 'Ma', 'Sha', 'Dha'],
 ['Ni', 'Re', 'Ni', 'Sha', 'Dha', 'Sa', 'Dha', 'Sha', 'Pa', 'Sa']]

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
import random

# Define the basic note set and raga scales
notes = ['Sa', 'Re', 'Ga', 'Ma', 'Pa', 'Dha', 'Ni', 'Sha']
raga_scales = {
    'Generic': notes,
    'Bhairav': ['Sa', 'Re', 'Ga', 'Ma', 'Pa', 'Dha', 'Ni'],
    'Bhopali': ['Sa', 'Re', 'Ga', 'Pa', 'Dha'],
    'Bageshree': ['Sa', 'Ga', 'Ma', 'Dha', 'Ni']
}

note_to_idx = {note: idx for idx, note in enumerate(notes)}
idx_to_note = {idx: note for note, idx in note_to_idx.items()}

# Step 1: Generate sequences
def generate_sequences(raga_name, seq_length=10, num_sequences=100):
    raga_notes = raga_scales[raga_name]
    sequences = []
    for _ in range(num_sequences):
        seq = [random.choice(raga_notes) for _ in range(seq_length)]
        sequences.append(seq)
    return sequences

# Step 2: Preprocess sequences (fixed)
def preprocess_sequences(sequences, seq_length):
    X, y = [], []
    for seq in sequences:
        # Only take sequences up to len(seq) - seq_length to ensure fixed length
        for i in range(len(seq) - seq_length):
            input_seq = [note_to_idx[note] for note in seq[i:i+seq_length]]
            output_note = note_to_idx[seq[i+seq_length]]
            X.append(input_seq)
            y.append(output_note)
    X = np.array(X)  # Now all sequences are of length seq_length
    y = np.array(y)
    return X, y

# Step 3: Build model
def build_model(vocab_size, seq_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=8, input_length=seq_length),
        SimpleRNN(64, return_sequences=False),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Step 4: Train model
def train_model(model, X, y, epochs=50, batch_size=32):
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)
    return model

# Step 5: Generate sequence
def generate_sequence(model, seed_seq, raga_name, length=20):
    raga_notes = raga_scales[raga_name]
    generated = seed_seq.copy()

    for _ in range(length):
        X_pred = np.array([generated[-5:]])  # Use last 5 notes
        pred = model.predict(X_pred, verbose=0)
        next_idx = np.argmax(pred[0])
        while idx_to_note[next_idx] not in raga_notes:
            pred[0][next_idx] = 0
            next_idx = np.argmax(pred[0])
        generated.append(next_idx)

    return [idx_to_note[idx] for idx in generated]

# Main function
def main(raga_name='Generic'):
    seq_length = 5
    num_sequences = 1000

    sequences = generate_sequences(raga_name, seq_length + 1, num_sequences)  # Ensure enough length
    X, y = preprocess_sequences(sequences, seq_length)

    vocab_size = len(notes)
    model = build_model(vocab_size, seq_length)
    model = train_model(model, X, y)

    seed_seq = [note_to_idx[note] for note in sequences[0][:seq_length]]
    generated_seq = generate_sequence(model, seed_seq, raga_name)
    print(f"Generated sequence for Raga {raga_name}: {' '.join(generated_seq)}")

# Run for all ragas
for raga in ['Generic', 'Bhairav', 'Bhopali', 'Bageshree']:
    print(f"\nTraining and generating for Raga {raga}")
    main(raga)


Training and generating for Raga Generic
Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.1342 - loss: 2.0802
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1769 - loss: 2.0696
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1567 - loss: 2.0642
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1668 - loss: 2.0587
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1717 - loss: 2.0371
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1690 - loss: 2.0389
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1681 - loss: 2.0427
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1753 - loss: 2.0291
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1901 - loss: 2.0255
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2073 - loss: 2.0260
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1791 - loss: 2.0278
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1983 - loss: 2.0235
